<a href="https://colab.research.google.com/github/vrajasekaran/gpt-micro/blob/main/gpt_micro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-31 14:31:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-31 14:31:23 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as inputFile:
  inputText = inputFile.read()

In [ ]:
print('input characters length:', len(inputText))
print('First 1000 chars:', inputText[0:1000])

input characters length: 1115394
First 1000 chars: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak thi

In [ ]:
#Build the vocabulary
vocabChars = sorted(list(set(inputText)))
print(''.join(vocabChars))
vocabSize = len(vocabChars)
print('vocab size:', vocabSize)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [ ]:
#Simple encode/decode
stringToEncode = 'My GPT works!'

stringToInteger = { ch:i for i,ch in enumerate(vocabChars) }
print(stringToInteger)
print('String to Encode:', stringToEncode)
encoder = lambda s: [stringToInteger[ch] for ch in s]
print(encoder(stringToEncode))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
String to Encode: My GPT works!
[25, 63, 1, 19, 28, 32, 1, 61, 53, 56, 49, 57, 2]


In [ ]:
integerToString = { i:ch for i,ch in enumerate(vocabChars)}
print(integerToString)
decoder = lambda l: ''.join([integerToString[i] for i in l])
print(decoder(encoder(stringToEncode)))

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
My GPT works!


In [ ]:
#ENcode the entire data
import torch
data = torch.tensor(encoder(inputText), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# Split Training and Validation datasets
n = int(0.9*len(data)) #90% for Training
train_data = data[:n]
validation_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
print('x:', x)
decoded_x = decoder(x.tolist())
print(f'Decoded x: {decoded_x}')

y = train_data[1:block_size+1]
print('y:', y)
decoded_y = decoder(y.tolist())
print(f'Decoded y: {decoded_y}')

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input is {context} the target: {target}')
  print(f'when input is {decoder(context.tolist())} the target: {decoder([target.item()])}')

x: tensor([18, 47, 56, 57, 58,  1, 15, 47])
Decoded x: First Ci
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])
Decoded y: irst Cit
when input is tensor([18]) the target: 47
when input is F the target: i
when input is tensor([18, 47]) the target: 56
when input is Fi the target: r
when input is tensor([18, 47, 56]) the target: 57
when input is Fir the target: s
when input is tensor([18, 47, 56, 57]) the target: 58
when input is Firs the target: t
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is First the target:  
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is First  the target: C
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is First C the target: i
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is First Ci the target: t


In [ ]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else validation_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:', xb.shape, xb)

for i in range(batch_size):
    print(f'input {i} is {decoder(xb[i].tolist())}')

#print(xb)
print('targets:', yb.shape)
for i in range(batch_size):
    print(f'input {i} is {decoder(yb[i].tolist())}')


inputs: torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
input 0 is Let's he
input 1 is for that
input 2 is nt that 
input 3 is MEO:
I p
targets: torch.Size([4, 8])
input 0 is et's hea
input 1 is or that 
input 2 is t that h
input 3 is EO:
I pa


In [ ]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")
    print(f'when input is {decoder(context.tolist())} the target: {decoder([target.item()])}')

when input is [24] the target: 43
when input is L the target: e
when input is [24, 43] the target: 58
when input is Le the target: t
when input is [24, 43, 58] the target: 5
when input is Let the target: '
when input is [24, 43, 58, 5] the target: 57
when input is Let' the target: s
when input is [24, 43, 58, 5, 57] the target: 1
when input is Let's the target:  
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is Let's  the target: h
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is Let's h the target: e
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is Let's he the target: a
when input is [44] the target: 53
when input is f the target: o
when input is [44, 53] the target: 56
when input is fo the target: r
when input is [44, 53, 56] the target: 1
when input is for the target:  
when input is [44, 53, 56, 1] the target: 58
when input is for  the target: t
when input is [44, 53, 56, 1, 58] the target: 46
when input is for t the t

In [ ]:
#BigramLanguageModel

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocabSize):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocabSize, vocabSize)

  def forward(self, idx, targets):
    logits = self.token_embedding_table(idx) # (B, T, C)

    return logits

m = BigramLanguageModel(vocabSize)